<a href="https://colab.research.google.com/github/saleh1312/nlp_projects/blob/main/Seq2Seq%20text%20translation%20keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import string
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy
import tensorflow as tf
nltk.download('all')

In [2]:
with open(r"small_vocab_en.txt", "r") as f:
    en_sent=f.read().split('\n')

with open(r"small_vocab_fr.txt", "r") as f:
    fr_sent=f.read().split('\n')


In [3]:
en_sent_clean=[]
for sent in en_sent:
    word_tok=word_tokenize(sent,'english')
    new_word=[]
    for token in word_tok:
        if token not in string.punctuation:
            new_word.append(token)
    en_sent_clean.append((' '.join(new_word)).lower()+' eos')

In [94]:
en_sent_clean[:3]

['new jersey is sometimes quiet during autumn and it is snowy in april eos',
 'the united states is usually chilly during july and it is usually freezing in november eos',
 'california is usually quiet during march and it is usually hot in june eos']

In [4]:
fr_sent_clean=[]
for sent in fr_sent:
    word_tok=word_tokenize(sent,'french')
    new_word=[]
    for token in word_tok:
        if token not in string.punctuation:
            new_word.append(token)
    fr_sent_clean.append('sos '+((' '.join(new_word)).lower()+' eos'))

In [95]:
fr_sent_clean[:3]

['sos new jersey est parfois calme pendant l automne et il est neigeux en avril eos',
 'sos les états-unis est généralement froid en juillet et il gèle habituellement en novembre eos',
 'sos california est généralement calme en mars et il est généralement chaud en juin eos']

In [5]:
toknizer_en  = tf.keras.preprocessing.text.Tokenizer()
toknizer_en.fit_on_texts(en_sent_clean)
sequences_en = toknizer_en.texts_to_sequences(en_sent_clean)

In [6]:
toknizer_fr  = tf.keras.preprocessing.text.Tokenizer()
toknizer_fr.fit_on_texts(fr_sent_clean)
sequences_fr = toknizer_fr.texts_to_sequences(fr_sent_clean)

In [96]:
sequences_en[:3]

[[18, 24, 1, 9, 68, 5, 40, 8, 4, 1, 56, 3, 45, 2],
 [6, 21, 22, 1, 10, 63, 5, 44, 8, 4, 1, 10, 52, 3, 46, 2],
 [23, 1, 10, 68, 5, 39, 8, 4, 1, 10, 69, 3, 35, 2]]

In [97]:
sequences_fr[:3]

[[2, 37, 36, 1, 10, 69, 39, 13, 26, 8, 5, 1, 114, 4, 52, 3],
 [2, 6, 34, 33, 1, 14, 21, 4, 51, 8, 5, 97, 71, 4, 53, 3],
 [2, 103, 1, 14, 69, 4, 47, 8, 5, 1, 14, 23, 4, 43, 3]]

In [7]:
frinput=[]
froutput=[]
for seq in sequences_fr:
    seqx=seq[:-1]
    seqy=seq[1:]
    frinput.append(seqx)
    froutput.append(seqy)

In [8]:
t1= max([len(w) for w in sequences_en])
t2= max([len(w) for w in sequences_fr])
maxlen=max([t1,t2])

In [9]:
pad_en=tf.keras.preprocessing.sequence.pad_sequences(sequences_en, padding='post',maxlen=maxlen)
pad_frinput=tf.keras.preprocessing.sequence.pad_sequences(frinput, padding='post',maxlen=maxlen)
pad_froutput=tf.keras.preprocessing.sequence.pad_sequences(froutput, padding='post',maxlen=maxlen)

In [10]:
hot_pad_fr=tf.keras.utils.to_categorical(pad_froutput)

In [46]:
input= tf.keras.Input((maxlen))
embd=tf.keras.layers.Embedding(len(toknizer_en.word_index)+1,7,input_length=maxlen)(input)
h0,h1,c=tf.keras.layers.LSTM(200,return_sequences=False,return_state=True)(embd)



input2= tf.keras.Input((maxlen))
decoder_embedding=tf.keras.layers.Embedding(len(toknizer_fr.word_index)+1,7,input_length=maxlen)(input2)

l2=tf.keras.layers.LSTM(200,return_sequences=True)(decoder_embedding, initial_state=[h1,c])

d=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(toknizer_fr.word_index)+1
                                                        ,activation='softmax'))(l2)
model=tf.keras.Model([input,input2],d)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 23)]         0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, 23, 7)        1393        ['input_9[0][0]']                
                                                                                                  
 embedding_9 (Embedding)        (None, 23, 7)        2429        ['input_10[0][0]']               
                                                                                            

In [56]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=
              tf.keras.optimizers.Adam(0.001))

In [ ]:
model.fit([pad_en,pad_frinput],hot_pad_fr,epochs=1000,batch_size=16)

In [93]:


r=6000
entext=en_sent_clean[r]
ground=fr_sent_clean[r]
print(entext)
print(ground[4:-4])

entoken=toknizer_en.texts_to_sequences([entext])
entokenpad=tf.keras.preprocessing.sequence.pad_sequences(entoken, padding='post',maxlen=maxlen)

frtext='sos'


i=0
while True:
  frtoken=toknizer_fr.texts_to_sequences([frtext])
  frtokenpad=tf.keras.preprocessing.sequence.pad_sequences(frtoken, padding='post',maxlen=maxlen)
  preds=model.predict([entokenpad,frtokenpad])
  index=np.argmax(preds[0,i])
  word=list(toknizer_fr.word_index.keys())[list(toknizer_fr.word_index.values()).index(index)]
  if word == 'eos':
    break
  frtext=frtext+' '+word
  i+=1
print(frtext[4:])

he likes grapes bananas and apples eos
il aime les raisins les bananes et les pommes
il aime les raisins les bananes et les pommes
